## TUTORIAL OF PYTORCH API 05 ==> CIFAR10
## WORK WITH CONVOLUTION NEURAL NETWORK

In [4]:
import os
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

## 0_1_Setup GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 0_2_Setup Tensorboard

In [5]:
writer = SummaryWriter('runs/mnist')


## 1_Dowload and Load dataset

In [7]:
import torch.utils


x_train = torchvision.datasets.CIFAR10(
    train=True,
    transform=torchvision.transforms.ToTensor(),
    root='./data/',
    download=True
)
x_test = torchvision.datasets.CIFAR10(
    train=False,
    transform=torchvision.transforms.ToTensor(),
    root='./data/',
    download=False
)

X_train = torch.utils.data.DataLoader(
    dataset=x_train,
    batch_size=64,
    shuffle=True,
)
X_test = torch.utils.data.DataLoader(
    dataset=x_test,
    batch_size=64,
    shuffle=False,
)

100%|██████████| 170498071/170498071 [00:48<00:00, 3512213.49it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data/


## 2_Explore dataset

In [9]:
print("Number of batch_size training examples: ", len(X_train))
print("Number of batch_size test examples: ", len(X_test))


Number of batch_size training examples:  782
Number of batch_size test examples:  157


In [10]:
print("Number of total training examples: ", 64*len(X_train))
print("Number of total test examples: ", 64*len(X_test))

Number of total training examples:  50048
Number of total test examples:  10048


In [12]:
example = iter(X_train)
example = next(example)
images, labels = example
print("Shape of examples in one batch: ", images.shape)
print("Shape of label examples in one batch: ", labels.shape)

Shape of examples in one batch:  torch.Size([64, 3, 32, 32])
Shape of label examples in one batch:  torch.Size([64])


In [21]:
print("One image shape: ",images[0].shape)
print("One label shape: ", labels[0].shape)
print("First label image: ", labels[0])
print("First image: \n",images[0])

One image shape:  torch.Size([3, 32, 32])
One label shape:  torch.Size([])
First label image:  tensor(4)
First image: 
 tensor([[[0.2039, 0.2588, 0.2627,  ..., 0.3294, 0.3647, 0.3882],
         [0.2431, 0.2471, 0.2588,  ..., 0.3333, 0.3451, 0.4235],
         [0.3020, 0.2784, 0.2784,  ..., 0.3804, 0.4157, 0.4941],
         ...,
         [0.5137, 0.4902, 0.4863,  ..., 0.9882, 0.9804, 0.9137],
         [0.5451, 0.5647, 0.5137,  ..., 0.9529, 0.9451, 0.8196],
         [0.4549, 0.4314, 0.4000,  ..., 0.6667, 0.6353, 0.5961]],

        [[0.1529, 0.2078, 0.2118,  ..., 0.2745, 0.2863, 0.2980],
         [0.1961, 0.1961, 0.2078,  ..., 0.2784, 0.2667, 0.3333],
         [0.2549, 0.2314, 0.2314,  ..., 0.3255, 0.3333, 0.4039],
         ...,
         [0.3843, 0.3686, 0.3686,  ..., 0.9882, 0.9725, 0.8941],
         [0.4235, 0.4471, 0.4118,  ..., 0.9529, 0.9373, 0.7922],
         [0.3569, 0.3451, 0.3216,  ..., 0.6667, 0.6275, 0.5725]],

        [[0.1176, 0.1725, 0.1765,  ..., 0.2000, 0.1922, 0.1961],
   

In [43]:
class_names = x_train.classes
print(len(class_names))
print(class_names)

10
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


## 3_Normalize dataset(Skip Step)

## 4_Plot to visualize some examples

In [44]:
def plot_examples(images, class_names, labels):
    plt.figure(figsize=(10,10))
    for i in range(16):
        plt.subplot(4,4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i].permute(1,2,0).numpy(), cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])

## 5_Build Model

In [50]:
class ConvolutionNN(torch.nn.Module):
    def __init__(self):
        super(ConvolutionNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32,kernel_size=3, padding=0)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32,kernel_size=3, padding=0)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(in_features=6*6*32, out_features=64)
        self.fc2 = torch.nn.Linear(in_features=64, out_features=10)


    def forward(self,x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.flatten(out)     #can use x.view(-1, n_c*n*n)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [53]:
model = ConvolutionNN().to(device)
print(summary(model, (3,32,32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             896
              ReLU-2           [-1, 32, 30, 30]               0
         MaxPool2d-3           [-1, 32, 15, 15]               0
            Conv2d-4           [-1, 32, 13, 13]           9,248
              ReLU-5           [-1, 32, 13, 13]               0
         MaxPool2d-6             [-1, 32, 6, 6]               0
           Flatten-7                 [-1, 1152]               0
            Linear-8                   [-1, 64]          73,792
              ReLU-9                   [-1, 64]               0
           Linear-10                   [-1, 10]             650
Total params: 84,586
Trainable params: 84,586
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.60
Params size (MB): 0.32
Estimated Tot

## 6_Select Hyperparameters for Model

In [61]:
epochs = 5
learning_rate = 0.001
n_steps = len(X_train)
n_steps

782

In [56]:
losses = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [59]:
for epoch in range(epochs):
    for i, (images, labels) in enumerate(X_train):
        model.train()
        images = images.to(device)
        labels = labels.to(device)

        #forward training
        outputs = model(images)
        loss = losses(outputs, labels)

        #backward training
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if(i+1) % 100 == 0:
            print(f"epoch {epoch+1} / {epochs}, step {i} / {n_steps}, loss = {loss.item():.4f}")
print("Finish Training")

Finish Training


## 7_Evaluation

In [67]:
batch_size = 64
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(class_names))]
    n_class_sample = [0 for i in range(len(class_names))]
    for i, (images, labels) in enumerate(X_test):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        #print(f"Prediction at batch {i}-th {predictions}")
        n_samples += len(labels)
        n_correct += (predictions == labels).sum().item()
        if len(labels) == 64:
            for j in range(batch_size):
                label = labels[j]
                pred = predictions[j]
                if(pred == label):
                    n_class_correct[label] += 1
                n_class_sample[label] += 1
        else:
            for j in range(len(labels)):
                label = labels[j]
                pred = predictions[j]
                if(pred == label):
                    n_class_correct[label] += 1
                n_class_sample[label] += 1
accuracy = 100*n_correct / n_samples
print("Accuracy of Model: ", accuracy)

for i in range(len(class_names)):
    accuracy = 100*n_class_correct[i] / n_class_sample[i]
    print(f"Accuracy of Class [{i}]: {accuracy}")

Accuracy of Model:  61.93
Accuracy of Class [0]: 62.6
Accuracy of Class [1]: 74.4
Accuracy of Class [2]: 63.9
Accuracy of Class [3]: 44.0
Accuracy of Class [4]: 39.8
Accuracy of Class [5]: 52.5
Accuracy of Class [6]: 78.6
Accuracy of Class [7]: 64.1
Accuracy of Class [8]: 82.6
Accuracy of Class [9]: 56.8
